In [1]:
from IPython.display import Markdown, display

In [2]:
def printmd(string):
    display(Markdown(string))

## Tomographic Reconstruction (Savu)

In [4]:
print("Tomographic reconstruction has been performed using Savu.\n")

print("Please cite:")
printmd("1. Kazantsev, Daniil and Wadeson, Nicola and Basham, Mark. "
        "High Performance Savu Software for Fast 3D Model-Based Iterative Reconstruction of Large Data at Diamond Light Source, "
        "Available at SSRN: <https://ssrn.com/abstract=3955840> or <http://dx.doi.org/10.2139/ssrn.3955840>\n"
)

Tomographic reconstruction has been performed using Savu.

Please cite:


1. Kazantsev, Daniil and Wadeson, Nicola and Basham, Mark. High Performance Savu Software for Fast 3D Model-Based Iterative Reconstruction of Large Data at Diamond Light Source, Available at SSRN: <https://ssrn.com/abstract=3955840> or <http://dx.doi.org/10.2139/ssrn.3955840>
